<a href="https://colab.research.google.com/github/AP2324/trabajo-equipo-9/blob/main/02_data_owner_review_approve_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import syft as sy
from syft.service.request.request import RequestStatus

print(f"{sy.__version__ = }")

sy.__version__ = '0.8.3'


In [ ]:
# Lanza un nodo de PySyft llamado 'mnist-domain' en modo de desarrollo.
node = sy.orchestra.launch(name="mnist-domain", dev_mode=True)

# El DO inicia sesión en el nodo para comenzar el proceso de revisión.
root_client = node.login(email="info@openmined.org", password="changethis")

Staging Protocol Changes...
Data Migrated to latest version !!!
Logged into <mnist-domain: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

## 1. DO reviews the submitted project and code

In [ ]:
# Visualiza los proyectos presentados al DO.
root_client.projects

In [ ]:
# Accede a las solicitudes del primer proyecto.
requests = root_client.projects[0].requests
requests

[syft.service.request.request.Request]

In [ ]:
# Asegura que hay una solicitud pendiente para revisión.
assert len(requests) == 1

In [ ]:
request = requests[0]
assert request.status == RequestStatus.PENDING
request

syft.service.request.request.Request

In [ ]:
change = request.changes[0]
change

```python
class UserCodeStatusChange:
  id: bf498f79178849bab5e3fcbf8b900e11
  function: mnist_3_linear_layers
  input_policy_type: ExactMatch
  output_policy_type: OutputPolicyExecuteOnce
  approved: False
```

#### Inspección del Código Enviado

In [ ]:
# Obtiene una referencia al objeto de código del usuario.
user_code = request.changes[0].link

# Muestra el código enviado para revisión.
user_code.show_code


```python
@sy.syft_function(input_policy=sy.ExactMatch(mnist_images=mock_images_ptr, mnist_labels=mock_labels_ptr),
                  output_policy=sy.SingleExecutionExactOutput())
# Definición de la función de entrenamiento del modelo
def mnist_3_linear_layers(mnist_images, mnist_labels):
    # Importación de los paquetes necesarios para la construcción y entrenamiento del modelo
    import jax #jax es una biblioteca diseñada para cálculos numéricos de alta velocidad
    from jax.example_libraries import stax #submódulo para construir y entrenar la red neuronal
    from jax.example_libraries.stax import Dense, Relu, LogSoftmax
    import time
    from jax.example_libraries import optimizers
    import itertools
    import jax.numpy as jnp # Importar el módulo numpy compatible con JAX
    import numpy.random as npr
    from jax import jit, grad, random

    # Definición de la arquitectura de la red neuronal utilizando capas densas y funciones de activación
    #stax.serial toma como argumentos las capas que compondrán la red, y las "apila" en el orden en que se proporcionan, de modo que la salida de una capa se convierte en la entrada de la siguiente. 
    init_random_params, predict = stax.serial( 
        Dense(1024), Relu,     # Capa densa o completamente conectada con 1024 neuronas
        Dense(1024), Relu,     # Relu es una funcion no lineal definida como f(x) = max(0, x) 
       #La función de activación LogSoftmax se aplica a la salida de la última capa densa. Convierte los logits en probabilidades logarítmicas 
        Dense(10), LogSoftmax) # capa densa, pero con 10 unidades, correspondientes a las 10 clases posibles de los dígitos MNIST (0 a 9)

    # Inicialización de los parámetros del modelo de forma aleatoria
    rng = random.PRNGKey(0)
    _, init_params = init_random_params(rng, (-1, 784)) # Inicia de manera aleatoria los parametros de la red antes de empezar el entrenamiento 

    # Número de pasadas completas sobre el conjunto de datos durante el entrenamiento.
    num_epochs = 10

    # Número de muestras por lote para actualizar los parámetros del modelo.
    batch_size = 4

    # Número total de imágenes en el conjunto de datos de entrenamiento.
    num_train = mnist_images.shape[0]

    # Cálculo del número de lotes completos y si hay un lote "incompleto" al final.
    num_complete_batches, leftover = divmod(num_train, batch_size)

    # El número total de lotes es el número de lotes completos más uno si hay un lote "incompleto".
    num_batches = num_complete_batches + bool(leftover)

    # Tamaño del paso para la actualización de los parámetros en el optimizador.
    step_size = 0.001

    # Parámetro 'mass' del optimizador de momentum que ayuda a suavizar las actualizaciones de los parámetros.
    momentum_mass = 0.9

    
    # Establecimiento de hiperparámetros como número de épocas, tamaño del lote, etc.
    num_epochs = 10 # Número de muestras por lote para actualizar los parámetros del modelo. 
    batch_size = 4  # Número total de imágenes en el conjunto de datos de entrenamiento..
    num_train = mnist_images.shape[0]  # Número total de imágenes en el conjunto de datos de entrenamiento.
    num_complete_batches, leftover = divmod(num_train, batch_size) # Cálculo del número de lotes completos y si hay un lote "incompleto" al final.
    num_batches = num_complete_batches + bool(leftover) # El número total de lotes es el número de lotes completos más uno si hay un lote "incompleto".
    step_size = 0.001 # Tamaño del paso para la actualización de los parámetros en el optimizador.
    momentum_mass = 0.9 # Parámetro 'mass' del optimizador de momentum que ayuda a suavizar las actualizaciones de los parámetros.

    # Inicialización del optimizador, en este caso, utilizando momentum
    opt_init, opt_update, get_params = optimizers.momentum(step_size, mass=momentum_mass)
    opt_state = opt_init(init_params)

    # Definición de la función de actualización del modelo para una iteración de entrenamiento
    @jit #Este decorador de JAX compila la función para mejorar su rendimiento mediante la compilación Just-In-Time
    def update(i, opt_state, batch):
        params = get_params(opt_state)
        return opt_update(i, grad(loss)(params, batch), opt_state)

    # Función para generar lotes de datos de forma aleatoria
    def data_stream():
        rng = npr.RandomState(0)
        while True:
            perm = rng.permutation(num_train)
            for i in range(num_batches):
                batch_idx = perm[i * batch_size:(i + 1) * batch_size]
                yield mnist_images[batch_idx], mnist_labels[batch_idx]

    # Definición de la función de pérdida
    def loss(params, batch):
        inputs, targets = batch
        preds = predict(params, inputs)
        return -jnp.mean(jnp.sum(preds * targets, axis=1))

    # Función para calcular la precisión del modelo
    def accuracy(params, batch):
        inputs, targets = batch
        target_class = jnp.argmax(targets, axis=1)
        predicted_class = jnp.argmax(predict(params, inputs), axis=1)
        return jnp.mean(predicted_class == target_class)

    # Proceso de entrenamiento, iterando sobre el número de épocas y actualizando el modelo en cada iteración
    batches = data_stream()
    train_accs = []
    print("\nStarting training...")

    # Defino 'itercount' 
    itercount = itertools.count(start=0)

    for epoch in range(num_epochs):
        start_time = time.time()
        for _ in range(num_batches):
           # 'next(itercount)' proporciona un contador único para cada actualización
            opt_state = update(next(itercount), opt_state, next(batches))
    epoch_time = time.time() - start_time
    params = get_params(opt_state)
    train_acc = accuracy(params, (mnist_images, mnist_labels))
    print(f"Epoch {epoch} in {epoch_time:0.2f} sec")
    print(f"Training set accuracy {train_acc}")
    train_accs.append(train_acc)

    # Retornar las precisiónes de entrenamiento y los parámetros del modelo después del entrenamiento
    return train_accs, params

```

#### Verificación de los Activos de Datos

In [ ]:
# Asegura que el código enviado incluye dos activos de datos (imágenes y etiquetas ficticias).
assert len(user_code.assets) == 2
user_code.assets

[syft.service.dataset.dataset.Asset, syft.service.dataset.dataset.Asset]

In [ ]:
# Accede a los datos ficticios para su revisión.
mock_images = user_code.assets[0].mock
print(f"{mock_images.shape = }")  # Muestra la forma de las imágenes ficticias.
mock_labels = user_code.assets[1].mock
print(f"{mock_labels.shape = }")  # Muestra la forma de las etiquetas ficticias.

mock_images.shape = (1000, 784)
mock_labels.shape = (1000, 10)


#### Ejecución del Código en Datos Ficticios por el DO

In [ ]:
# Ejecuta la función del usuario en datos ficticios para asegurar que todo funciona correctamente.
users_function = user_code.unsafe_function  # Accede a la función del usuario.
users_function

SyftWarning: This code was submitted by a User and could be UNSAFE.

<function syft.service.code.user_code.UserCode.unsafe_function.<locals>.wrapper(*args: 'Any', **kwargs: 'Any') -> 'Callable'>

In [ ]:
mock_train_accs, mock_params = users_function(mnist_images=mock_images, mnist_labels=mock_labels)


Starting training...
Epoch 9 in 4.25 sec
Training set accuracy 0.9250000715255737


In [ ]:
# Verifica que los resultados sean listas, como se espera.
assert isinstance(mock_train_accs, list)
mock_train_accs

[Array(0.9250001, dtype=float32)]

In [ ]:
# Verifica que los resultados sean listas, como se espera.
assert isinstance(mock_params, list)
mock_params

[(Array([[ 0.00545607, -0.05433224, -0.02506988, ..., -0.00098425,
          -0.03896995, -0.03779807],
         [-0.03839426,  0.05401544, -0.03861534, ...,  0.02070526,
           0.03663716, -0.02790845],
         [-0.05057753, -0.0738507 ,  0.07722926, ..., -0.02943752,
           0.01401689,  0.02046063],
         ...,
         [-0.03310735,  0.02437904,  0.00343548, ...,  0.03456359,
          -0.04919678, -0.03409548],
         [ 0.01774163, -0.00695568, -0.02793701, ...,  0.02670364,
          -0.00640533, -0.00805017],
         [ 0.03140757,  0.00657581,  0.00111776, ...,  0.02953581,
          -0.04007518,  0.06783257]], dtype=float64),
  Array([-0.00956236,  0.00723611,  0.00352483, ..., -0.01150145,
         -0.0257812 ,  0.01425009], dtype=float64)),
 (),
 (Array([[-0.00700192,  0.05190444, -0.02324766, ..., -0.02960721,
           0.00090052, -0.06492114],
         [ 0.04431988, -0.04059733, -0.02139692, ...,  0.05273425,
           0.00951498, -0.02368151],
         [-0.0188665 , -0.01252971,  0.02343362, ...,  0.03167466,
          -0.00548374,  0.00037753],
         ...,
         [-0.00746664,  0.00817988,  0.02925574, ...,  0.0156354 ,
          -0.02083614,  0.00298294],
         [-0.06270901, -0.05858192, -0.06206076, ..., -0.01127014,
          -0.06813121, -0.01481143],
         [-0.00438562,  0.04385707,  0.05901627, ..., -0.03107409,
          -0.02119039,  0.01733593]], dtype=float64),
  Array([ 0.0010008 , -0.00698441,  0.00262431, ...,  0.00668819,
         -0.01466187,  0.00486839], dtype=float64)),
 (),
 (Array([[ 0.00686753, -0.00802625,  0.05345474, ..., -0.02885558,
          -0.0246781 , -0.00026447],
         [-0.08211078, -0.02190682, -0.00674321, ...,  0.08242337,
          -0.02829694,  0.02195659],
         [ 0.07368974,  0.00861787, -0.05080931, ..., -0.05426908,
           0.07653555, -0.06038157],
         ...,
         [ 0.0646016 , -0.08493688, -0.1060349 , ..., -0.0531944 ,
          -0.08744076, -0.01365703],
         [ 0.02223047,  0.0197634 ,  0.01210248, ...,  0.02972855,
           0.02413002, -0.04433617],
         [-0.0416151 , -0.0502627 ,  0.0220279 , ...,  0.0068896 ,
           0.14579427, -0.06054653]], dtype=float64),
  Array([ 0.01507639,  0.01168913, -0.00270039,  0.00575929,  0.00355094,
         -0.00263394, -0.02147321, -0.01024549, -0.00644285,  0.01831268],      dtype=float64)),
 ()]

## 2.El DO ejecuta el código enviado en datos privados y luego deposita los resultados en el dominio para que el DS pueda recuperarlos.

In [ ]:
# Accede a los datos reales asociados con los activos.
private_images = user_code.assets[0].data
print(f"{private_images.shape = }")
private_labels = user_code.assets[1].data
print(f"{private_labels.shape = }")

private_images.shape = (1000, 784)
private_labels.shape = (1000, 10)


In [ ]:
# Ejecuta la función del usuario en datos reales.
train_accs, params = users_function(mnist_images=private_images, mnist_labels=private_labels)


Starting training...
Epoch 9 in 4.46 sec
Training set accuracy 0.9970000386238098


In [ ]:
#Verifica que los resultados sean listas.
assert isinstance(train_accs, list)
train_accs

[Array(0.99700004, dtype=float32)]

In [ ]:
#Verifica que los resultados sean listas.
assert isinstance(params, list)
params

[(Array([[ 0.00147607, -0.04636604, -0.02811676, ..., -0.00285015,
          -0.03563077, -0.038009  ],
         [-0.03454075,  0.0489547 , -0.04267067, ...,  0.0122015 ,
           0.04534056, -0.02854517],
         [-0.05238249, -0.06238068,  0.07483938, ..., -0.03070428,
           0.00975748,  0.02083589],
         ...,
         [-0.02724382,  0.02700684, -0.00539466, ...,  0.02555376,
          -0.04593473, -0.03454394],
         [ 0.01646777, -0.00847972, -0.02544667, ...,  0.01747823,
          -0.00734709, -0.00751594],
         [ 0.02964841,  0.00608065,  0.00581593, ...,  0.02426394,
          -0.03714505,  0.06703147]], dtype=float64),
  Array([-0.00647509,  0.01645931,  0.02217218, ..., -0.01824659,
         -0.01759022,  0.01898984], dtype=float64)),
 (),
 (Array([[-0.01054671,  0.05306379, -0.02392295, ..., -0.03089813,
           0.00094881, -0.06276244],
         [ 0.04584206, -0.03480456, -0.0257966 , ...,  0.04197395,
           0.00931185, -0.02621233],
         [-0.0196184 , -0.01548337,  0.02462837, ...,  0.01735921,
          -0.00463263,  0.01383693],
         ...,
         [-0.00517053,  0.00461716,  0.03038958, ...,  0.00786606,
          -0.02100694,  0.00618721],
         [-0.06284097, -0.05657079, -0.05905942, ..., -0.01964503,
          -0.06744551, -0.01943832],
         [-0.0034698 ,  0.04567881,  0.05527141, ..., -0.03437851,
          -0.0211427 ,  0.01578522]], dtype=float64),
  Array([ 0.00012847,  0.00191936,  0.00429164, ..., -0.00244934,
         -0.01339345, -0.0112778 ], dtype=float64)),
 (),
 (Array([[-0.00501813, -0.00699326,  0.0153345 , ..., -0.0348444 ,
          -0.03313165,  0.03534848],
         [-0.01602623, -0.04143519, -0.03394898, ...,  0.12365393,
          -0.06600836,  0.06653416],
         [ 0.09111957,  0.04596651, -0.06106429, ..., -0.07793583,
           0.13748278, -0.08908825],
         ...,
         [ 0.08923355, -0.09285264, -0.05303064, ..., -0.08844142,
          -0.04708981, -0.06573843],
         [ 0.02211408,  0.01966383,  0.01300406, ...,  0.03041883,
           0.02246484, -0.0454883 ],
         [ 0.03114939, -0.08224195,  0.02024963, ...,  0.02218605,
           0.11749071, -0.04671701]], dtype=float64),
  Array([-0.03916654,  0.05372111, -0.02324924, -0.00031544,  0.03098593,
          0.03628296, -0.00735236,  0.02930066, -0.07747608,  0.00816158],      dtype=float64)),
 ()]

In [ ]:
# Acepta la solicitud y deposita los resultados en el dominio para que el DS pueda recuperarlos.
res = request.accept_by_depositing_result((train_accs, params))

Request approved for domain mnist-domain
Request approved for domain mnist-domain


In [ ]:
assert isinstance(res, sy.SyftSuccess)
res

SyftSuccess: Request d52bc75521fd40298e9cbc37652b581e changes applied

### 📓 Now switch to the [second DS's notebook](./03-data-scientist-download-results.ipynb)